In [1]:
import pandas as pd
import numpy as np
import _pickle as cPickle

In [2]:
unpickled_dict = cPickle.load(open("arr_hispanics.pkl", "rb"))
print(unpickled_dict["arr"].shape)
print(len(unpickled_dict["img_path"]))

(184, 4, 512)
184


In [3]:
# check with predictions and patient identifier
predictions = pd.read_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/preds_hispanic0622.csv")
predictions # same length as the first axis in the composite numpy array, so the axis and indices can be used for merging

,patient_exam_id,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk
0,18276219\t18276219.2O04729,0.067194,0.092891,0.100460,0.121022,0.128137
1,18276219\t18276219.2O04730,0.017476,0.027304,0.035877,0.042368,0.049386
2,18276219\t18276219.2O04727,0.013118,0.021093,0.028833,0.033885,0.040194
3,18276219\t18276219.2O04731,0.043225,0.062224,0.071790,0.086112,0.094150
4,16722089\t16722089.2O15338,0.002532,0.005728,0.009546,0.011385,0.014396
...,...,...,...,...,...,...
179,93924458\t93924458.2O13906,0.001702,0.004172,0.007314,0.009791,0.012468
180,93924458\t93924458.2O33884,0.001879,0.005273,0.008903,0.013110,0.016466
181,93924458\t93924458.2O33883,0.001730,0.004441,0.007707,0.010783,0.013669
182,97681331\t97681331.2O11263,0.002121,0.006376,0.010450,0.015570,0.019388


In [4]:
arr = unpickled_dict["arr"]

In [5]:
# reshape such that the four views are set along one array
arr = arr.reshape(len(arr), 4 * 512)

In [6]:
print(arr.shape)
print(arr.ndim)
# now 2-dimensional array, the 0th axis is the index that will be used to merge with Mirai output

(184, 2048)
2


In [7]:
arrdf = pd.DataFrame(arr) #184 rows, one exam per row, and 20148 columns for the representations

In [8]:
preddf = pd.DataFrame(predictions)

In [9]:
preddf

,patient_exam_id,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk
0,18276219\t18276219.2O04729,0.067194,0.092891,0.100460,0.121022,0.128137
1,18276219\t18276219.2O04730,0.017476,0.027304,0.035877,0.042368,0.049386
2,18276219\t18276219.2O04727,0.013118,0.021093,0.028833,0.033885,0.040194
3,18276219\t18276219.2O04731,0.043225,0.062224,0.071790,0.086112,0.094150
4,16722089\t16722089.2O15338,0.002532,0.005728,0.009546,0.011385,0.014396
...,...,...,...,...,...,...
179,93924458\t93924458.2O13906,0.001702,0.004172,0.007314,0.009791,0.012468
180,93924458\t93924458.2O33884,0.001879,0.005273,0.008903,0.013110,0.016466
181,93924458\t93924458.2O33883,0.001730,0.004441,0.007707,0.010783,0.013669
182,97681331\t97681331.2O11263,0.002121,0.006376,0.010450,0.015570,0.019388


In [10]:
def prep_results_input(results_path, input_path):
    ''' 
    takes in mirai prediction path from results_path, and the input csv path
    from input_path
    outputs dataframe to be used in survival analysis to combined_path
    '''
    input = pd.read_csv(input_path)
    results = pd.read_csv(results_path)
    input['study_id'] = input['patient_id']
    results['exam_id']= results['patient_exam_id'].str.split("\t", n=1, expand = True)[1] #split out exam_id to merge with input sheet
    results = results.merge(input, how='left', on='exam_id', indicator=True)
    columns_to_keep= ["exam_id", "1_year_risk", "2_year_risk", "3_year_risk", "4_year_risk", "5_year_risk", "case", "years_to_cancer", "years_to_last_followup", "patient_id"]
    results = results[columns_to_keep]
    results["study_id"] = results["patient_id"]
    print("there are {} duplicated rows and {} unduplicated rows \n{} total rows".\
    format(results.duplicated().sum(), (~results.duplicated()).sum(), len(results)))
    print('*'*20)
    print("dropping duplicate rows; so only one entry per exam")
    results = results.drop_duplicates() 
    return results 

In [11]:
results_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/preds_hispanic0622.csv"
input_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/hispanic_set_1112.csv"

In [12]:
pred_cleaned = prep_results_input(results_path, input_path)

there are 609 duplicated rows and 184 unduplicated rows 
793 total rows
********************
dropping duplicate rows; so only one entry per exam


In [13]:
pred_cleaned

,exam_id,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk,case,years_to_cancer,years_to_last_followup,patient_id,study_id
0,18276219.2O04729,0.067194,0.092891,0.100460,0.121022,0.128137,False,100,5,18276219,18276219
6,18276219.2O04730,0.017476,0.027304,0.035877,0.042368,0.049386,False,100,4,18276219,18276219
11,18276219.2O04727,0.013118,0.021093,0.028833,0.033885,0.040194,False,100,8,18276219,18276219
15,18276219.2O04731,0.043225,0.062224,0.071790,0.086112,0.094150,False,100,3,18276219,18276219
19,16722089.2O15338,0.002532,0.005728,0.009546,0.011385,0.014396,False,100,3,16722089,16722089
...,...,...,...,...,...,...,...,...,...,...,...
772,93924458.2O13906,0.001702,0.004172,0.007314,0.009791,0.012468,False,100,3,93924458,93924458
776,93924458.2O33884,0.001879,0.005273,0.008903,0.013110,0.016466,False,100,1,93924458,93924458
781,93924458.2O33883,0.001730,0.004441,0.007707,0.010783,0.013669,False,100,1,93924458,93924458
785,97681331.2O11263,0.002121,0.006376,0.010450,0.015570,0.019388,False,100,2,97681331,97681331


In [14]:
pred_cleaned.reset_index(inplace=True)
pred_cleaned

,index,exam_id,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk,case,years_to_cancer,years_to_last_followup,patient_id,study_id
0,0,18276219.2O04729,0.067194,0.092891,0.100460,0.121022,0.128137,False,100,5,18276219,18276219
1,6,18276219.2O04730,0.017476,0.027304,0.035877,0.042368,0.049386,False,100,4,18276219,18276219
2,11,18276219.2O04727,0.013118,0.021093,0.028833,0.033885,0.040194,False,100,8,18276219,18276219
3,15,18276219.2O04731,0.043225,0.062224,0.071790,0.086112,0.094150,False,100,3,18276219,18276219
4,19,16722089.2O15338,0.002532,0.005728,0.009546,0.011385,0.014396,False,100,3,16722089,16722089
...,...,...,...,...,...,...,...,...,...,...,...,...
179,772,93924458.2O13906,0.001702,0.004172,0.007314,0.009791,0.012468,False,100,3,93924458,93924458
180,776,93924458.2O33884,0.001879,0.005273,0.008903,0.013110,0.016466,False,100,1,93924458,93924458
181,781,93924458.2O33883,0.001730,0.004441,0.007707,0.010783,0.013669,False,100,1,93924458,93924458
182,785,97681331.2O11263,0.002121,0.006376,0.010450,0.015570,0.019388,False,100,2,97681331,97681331


In [15]:
#now can merge index with data containing arrays 
# by exam_id: race, tumor subtype, old versus new exams 

columns_to_keep = ['exam_id', 'case', 'patient_id']
pc = pred_cleaned[columns_to_keep]

In [16]:
arr = pd.merge(pc, arrdf, left_index=True, right_index=True)

In [17]:
arr

,exam_id,case,patient_id,0,1,2,3,4,5,6,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,18276219.2O04729,False,18276219,-0.000077,0.492520,-0.000158,-0.019214,-0.016856,-0.002422,-0.009003,...,0.026122,-0.003400,-0.000820,-0.004019,0.214487,-0.000171,-0.018130,0.080727,-0.000005,-0.016484
1,18276219.2O04730,False,18276219,-0.000067,0.771432,-0.000171,-0.019068,-0.016744,-0.002416,-0.008959,...,0.073838,-0.003400,-0.000820,-0.004021,0.184625,-0.000169,-0.018134,0.164037,-0.000005,-0.016488
2,18276219.2O04727,False,18276219,-0.000072,0.694394,-0.000164,-0.019148,-0.016806,-0.002420,-0.008983,...,0.113081,-0.003399,-0.000818,-0.004023,0.165355,-0.000166,-0.018178,0.221193,-0.000005,-0.016497
3,18276219.2O04731,False,18276219,-0.000076,0.605234,-0.000159,-0.019204,-0.016851,-0.002422,-0.009000,...,0.007507,-0.003401,-0.000821,-0.004017,0.217291,-0.000175,-0.018073,0.064854,-0.000005,-0.016476
4,16722089.2O15338,False,16722089,-0.000078,0.156557,-0.000156,-0.019199,-0.016849,-0.002423,-0.009004,...,0.319006,-0.003395,-0.000815,-0.004033,0.050166,-0.000154,-0.018370,0.546118,-0.000005,-0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,93924458.2O13906,False,93924458,-0.000078,0.158127,-0.000156,-0.019202,-0.016851,-0.002423,-0.009005,...,0.103438,-0.003399,-0.000819,-0.004023,0.173545,-0.000166,-0.018193,0.198812,-0.000005,-0.016496
180,93924458.2O33884,False,93924458,-0.000077,0.386667,-0.000158,-0.019183,-0.016831,-0.002422,-0.008996,...,0.066303,-0.003400,-0.000819,-0.004022,0.197622,-0.000167,-0.018262,0.109118,-0.000005,-0.016494
181,93924458.2O33883,False,93924458,-0.000077,0.364021,-0.000158,-0.019179,-0.016832,-0.002422,-0.008997,...,0.183118,-0.003397,-0.000817,-0.004028,0.131645,-0.000159,-0.018294,0.310170,-0.000005,-0.016513
182,97681331.2O11263,False,97681331,-0.000075,0.488419,-0.000160,-0.019156,-0.016812,-0.002421,-0.008988,...,0.012580,-0.003401,-0.000820,-0.004019,0.225750,-0.000171,-0.018234,0.020602,-0.000005,-0.016486


In [18]:
arr.case.value_counts()

False    164
True      20
Name: case, dtype: int64

In [19]:
arr.to_csv('hispanic_arrays.csv')